<a href="https://colab.research.google.com/github/Rainwoorimforest/pytorch-study/blob/main/src/CNN_CIFAR10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
from torch import nn
from torchvision import datasets
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader, random_split
import numpy as np

In [ ]:
DEVICE = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(DEVICE, torch.__version__)

cpu 2.6.0+cu124


In [ ]:
train_dataset = datasets.CIFAR10(root="data/CIFAR10_DATA/", train = True, download=True,
                                 transform=transforms.ToTensor())
test_dataset = datasets.CIFAR10(root="data/CIFAR10_DATA/", train = False, download=True,
                                 transform=transforms.ToTensor())

100%|██████████| 170M/170M [00:03<00:00, 50.4MB/s]


In [ ]:
print(len(train_dataset)) #batch_size를 알기 위해서

50000


In [ ]:
train_dataset_size = int(len(train_dataset) * 0.85)
valid_dataset_size = len(train_dataset) - train_dataset_size
train_dataset, valid_dataset = random_split(train_dataset, [train_dataset_size, valid_dataset_size])

In [ ]:
print(len(train_dataset))
print(len(valid_dataset))
print(len(test_dataset))

42500
7500
10000


In [ ]:
BATCH_SIZE  = 64

train_dataset_loader = DataLoader(dataset = train_dataset, batch_size = BATCH_SIZE, shuffle=True)
valid_dataset_loader = DataLoader(dataset = valid_dataset, batch_size = BATCH_SIZE, shuffle=True)
test_dataset_loader = DataLoader(dataset = test_dataset, batch_size = BATCH_SIZE, shuffle=True)

In [ ]:
class MyCNNModel(nn.Module):
    def __init__(self):
        super().__init__()

        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1)

        self.pooling = nn.MaxPool2d(kernel_size=2, stride=2)

        self.fc1 = nn.Linear(8 * 8 * 64, 128)
        self.fc2 = nn.Linear(128, 10)

        self.dropout25 = nn.Dropout(p=0.25)
        self.dropout50 = nn.Dropout(p=0.5)

    def forward(self, data):

        data = self.conv1(data)
        data = torch.relu(data)
        data = self.pooling(data)
        data = self.dropout25(data)

        data = self.conv2(data)
        data = torch.relu(data)
        data = self.pooling(data)
        data = self.dropout25(data)

        data = data.view(-1, 8 * 8 * 64)

        data = self.fc1(data)
        data = torch.relu(data)
        data = self.dropout50(data)

        logits = self.fc2(data)

        return logits

In [ ]:
model = MyCNNModel().to(DEVICE)

loss_function = nn.CrossEntropyLoss()

optimizer = torch.optim.Adam(model.parameters(), lr = 1e-3)

In [ ]:
print(len(dataloader))

NameError: name 'dataloader' is not defined

In [ ]:
def model_train(dataloader, model, loss_function, optimizer):

    model.train()

    train_loss_sum = train_correct = train_total = 0

    total_train_batch = len(dataloader)
    print(len(dataloader))

    for images, labels in dataloader:

        x_train = images.to(DEVICE)
        y_train = labels.to(DEVICE)

        outputs = model(x_train)
        loss = loss_function(outputs, y_train)

        optimizer.zero_grad() # 이전 배치에서 계산된 기울기(gradient)를 초기화
        loss.backward() # 역전파 w 등 하이퍼파라미터 기울기 누적
        optimizer.step()

        train_loss_sum += loss.item()

        train_total += y_train.size(0)
        train_correct += ((torch.argmax(outputs, 1)==y_train)).sum().item()

    train_avg_loss = train_loss_sum / total_train_batch
    train_avg_accuracy = 100*train_correct / train_total

    return (train_avg_loss, train_avg_accuracy)

In [ ]:
def model_evaluate(dataloader, model, loss_function, optimizer):

    model.eval()

    with torch.no_grad():

        val_loss_sum = val_correct = val_total = 0

        total_val_batch = len(dataloader)

        for images, labels in dataloader:

            x_val = images.to(DEVICE)
            y_val = labels.to(DEVICE)

            outputs = model(x_val)
            loss = loss_function(outputs, y_val)

            val_loss_sum += loss.item()

            val_total += y_val.size(0)
            val_correct += ((torch.argmax(outputs, 1)==y_val)).sum().item()

        val_avg_loss = val_loss_sum / total_val_batch
        val_avg_accuracy = 100*val_correct / val_total

    return (val_avg_loss, val_avg_accuracy)

In [ ]:

def model_test(dataloader, model):

    model.eval()

    with torch.no_grad():

        test_loss_sum = test_correct = test_total = 0

        total_test_batch = len(dataloader)

        for images, labels in dataloader:

            x_test = images.to(DEVICE)
            y_test = labels.to(DEVICE)

            outputs = model(x_test)
            loss = loss_function(outputs, y_test)

            test_loss_sum += loss.item()

            test_total += y_test.size(0)
            test_correct += ((torch.argmax(outputs, 1)==y_test)).sum().item()

        test_avg_loss = test_loss_sum / total_test_batch
        test_avg_accuracy = 100*test_correct / test_total

        print('accuracy:', test_avg_accuracy)
        print('loss:', test_avg_loss)

In [ ]:
from datetime import datetime

train_loss_list = []
train_accuracy_list = []

val_loss_list = []
val_accuracy_list = []

start_time = datetime.now()

EPOCHS = 100

for epoch in range(EPOCHS):
   #==============  model train  ================
    train_avg_loss, train_avg_accuracy = model_train(train_dataset_loader, model, loss_function, optimizer)

    train_loss_list.append(train_avg_loss)
    train_accuracy_list.append(train_avg_accuracy)
    #=============================================

    #============  model evaluation  ==============
    val_avg_loss, val_avg_accuracy = model_evaluate(valid_dataset_loader, model, loss_function, optimizer)

    val_loss_list.append(val_avg_loss)
    val_accuracy_list.append(val_avg_accuracy)
    #============  model evaluation  ==============

    print('epoch:', '%02d' % (epoch + 1),
          'train loss =', '{:.3f}'.format(train_avg_loss), 'train acc =', '{:.3f}'.format(train_avg_accuracy),
          'val loss =', '{:.3f}'.format(val_avg_loss), 'val acc =', '{:.3f}'.format(val_avg_accuracy))

end_time = datetime.now()

print('elapsed time => ', end_time-start_time)

665
epoch: 01 train loss = 1.441 train acc = 47.866 val loss = 1.285 val acc = 53.253
665
epoch: 02 train loss = 1.326 train acc = 52.266 val loss = 1.184 val acc = 57.400
665
epoch: 03 train loss = 1.256 train acc = 54.741 val loss = 1.107 val acc = 61.093
665
epoch: 04 train loss = 1.199 train acc = 56.885 val loss = 1.056 val acc = 62.840
665
epoch: 05 train loss = 1.149 train acc = 59.024 val loss = 0.976 val acc = 65.227
665
epoch: 06 train loss = 1.111 train acc = 59.984 val loss = 0.957 val acc = 66.067
665
epoch: 07 train loss = 1.082 train acc = 61.358 val loss = 0.976 val acc = 65.880
665
epoch: 08 train loss = 1.056 train acc = 62.626 val loss = 0.931 val acc = 67.333
665
epoch: 09 train loss = 1.032 train acc = 63.351 val loss = 0.902 val acc = 68.573
665
epoch: 10 train loss = 1.010 train acc = 63.711 val loss = 0.893 val acc = 68.280
665
epoch: 11 train loss = 0.994 train acc = 64.546 val loss = 0.884 val acc = 69.173
665
epoch: 12 train loss = 0.975 train acc = 65.492 va

In [ ]:

import matplotlib.pyplot as plt
plt.figure(figsize=(10,5))

plt.subplot(1,2,1)
plt.title('Loss Trend')
plt.xlabel('epochs')
plt.ylabel('loss')
plt.grid()
plt.plot(train_loss_list, label='train')
plt.plot(val_loss_list, label='validation')
plt.legend()

plt.subplot(1,2,2)
plt.title('Accuracy Trend')
plt.xlabel('epochs')
plt.ylabel('accuracy')
plt.grid()
plt.plot(train_accuracy_list, label='train')
plt.plot(val_accuracy_list, label='validation')
plt.legend()

plt.show()